## 4 -> Customer Metrics

### Reading Data

In [33]:
import pandas as pd
import numpy as np

cleaned_df = pd.read_csv('../data/processed/cleaned_data.csv')
cleaned_df['TotalAmount'] = cleaned_df['Quantity'] * cleaned_df['UnitPrice']

### Calculating Customer Metrics

In [34]:
customer_metrics = cleaned_df.groupby(by='CustomerID', as_index=False)\
  .agg(
    TotalSpent = ('TotalAmount', 'sum'),
    OrderCount = ('InvoiceNo', 'nunique'),
    FirstInvoiceDate = ('InvoiceDate', 'min'),
    LastInvoiceDate =('InvoiceDate', 'max')
  )

In [35]:
# customer_metrics = df.groupby(by='CustomerID', as_index=False)\
#   .agg({
#     'InvoiceNo':'count', #total no of orders per customer
#     'TotalAmount':['sum','mean'], #total amount ordered and average order amount
#     'InvoiceDate': ['min', 'max'], #first and last purchase date
# }).round(2)
# customer_metrics.columns = ['order_count', 'total_spend', 'avg_order_value', 'first_purchase',  'last_purchase']

In [36]:
customer_metrics['LastInvoiceDate'] = pd.to_datetime(customer_metrics['LastInvoiceDate'])
customer_metrics['FirstInvoiceDate'] = pd.to_datetime(customer_metrics['FirstInvoiceDate'])

customer_metrics['CustomerLifespan'] = (customer_metrics['LastInvoiceDate'] - customer_metrics['FirstInvoiceDate']).dt.days

In [37]:
#using the maxinvoice date from the table instead of current date since the data is old
max_invoice_date = customer_metrics['LastInvoiceDate'].max()

customer_metrics['Recency'] = (max_invoice_date - customer_metrics['LastInvoiceDate']).dt.days

In [38]:
customer_metrics['CustomerLifespan'] = np.where(customer_metrics['CustomerLifespan'] == 0, \
                                                1, customer_metrics['CustomerLifespan'])

customer_metrics['PurchaseFrequency'] = (customer_metrics['OrderCount'] / (customer_metrics['CustomerLifespan'] / 30)
).round(2)

In [39]:
customer_metrics.head()

,CustomerID,TotalSpent,OrderCount,FirstInvoiceDate,LastInvoiceDate,CustomerLifespan,Recency,PurchaseFrequency
0,12346.0,77183.60,1,2011-01-18 10:01:00,2011-01-18 10:01:00,1,325,30.00
1,12347.0,4310.00,7,2010-12-07 14:57:00,2011-12-07 15:52:00,365,1,0.58
2,12348.0,1437.24,4,2010-12-16 19:09:00,2011-09-25 13:13:00,282,74,0.43
3,12349.0,1457.55,1,2011-11-21 09:51:00,2011-11-21 09:51:00,1,18,30.00
4,12350.0,294.40,1,2011-02-02 16:01:00,2011-02-02 16:01:00,1,309,30.00


### Exporting Data

In [40]:
customer_metrics.to_csv('../data/processed/customer_metrics.csv', index=False)